In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import zipfile
import os
from google.colab import drive



In [ ]:
drive.mount('/content/drive/')
checkpoint_path = "/content/drive/MyDrive/model_checkpoint_scratch_01.pth"

In [ ]:
data_dir = '/content/drive/My Drive/data/classification/dataset_CIFAR10.zip'

In [ ]:
!mkdir /content/dataset/
!unzip "/content/drive/My Drive/data/classification/dataset_CIFAR10.zip" -d "/content/dataset/"

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])



train_dataset = datasets.ImageFolder(root="/content/dataset/train", transform=train_transform)
test_dataset = datasets.ImageFolder(root="/content/dataset/validation", transform=test_transform)


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


class ImageClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(ImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

    def predict(self, x):
      return torch.softmax(self.forward(x), dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(train_dataset.classes)
criterion = nn.CrossEntropyLoss()


In [ ]:
def train_iteration(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    average_loss = 0
    average_accuracy = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        predictions = model(images)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        average_loss += loss.item()
        average_accuracy += (predictions.argmax(1) == labels).type(torch.float).sum().item()

    average_loss /= len(dataloader)
    average_accuracy /= size
    return {'average_loss': average_loss, 'average_accuracy': average_accuracy}

def test_iteration(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    average_loss = 0
    average_accuracy = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            predictions = model(images)
            average_loss += loss_fn(predictions, labels).item()
            average_accuracy += (predictions.argmax(1) == labels).type(torch.float).sum().item()
    average_loss /= num_batches
    average_accuracy /= size
    return {'average_loss': average_loss, 'average_accuracy': average_accuracy}

In [ ]:
model = ImageClassifier(num_classes=len(train_dataset.classes)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

try:
  checkpoint = torch.load(checkpoint_path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  start_epoch = checkpoint['epoch']
  print(f"Resuming training from epoch {start_epoch}")
except FileNotFoundError:
  print("No checkpoint found. Starting training from scratch.")
  start_epoch = 0

In [ ]:
import json

def save_metrics(epoch, train_results, test_results, filename="/content/drive/MyDrive/training_metrics_scratch_01.json"):
    try:
        with open(filename, 'r') as f:
            metrics = json.load(f)
    except FileNotFoundError:
        metrics = []

    metrics.append({
        'epoch': epoch + 1,
        'train_loss': train_results['average_loss'],
        'train_accuracy': train_results['average_accuracy'],
        'test_loss': test_results['average_loss'],
        'test_accuracy': test_results['average_accuracy']
    })

    with open(filename, 'w') as f:
        json.dump(metrics, f, indent=4)

In [ ]:
num_epochs = 30
for epoch in range(start_epoch, num_epochs):
  print(f"Epoch {epoch+1}\n-------------------------------")
  train_results = train_iteration(train_loader, model, criterion, optimizer)
  test_results = test_iteration(test_loader, model, criterion)
  save_metrics(epoch, train_results, test_results)
  print("Metrics saved to training_metrics_scratch_01.json")
  print(f"Train Loss: {train_results['average_loss']}, Train Accuracy: {train_results['average_accuracy']}")
  print(f"Test Loss: {test_results['average_loss']}, Test Accuracy: {test_results['average_accuracy']}")
  checkpoint = {
      'epoch': epoch + 1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'train_loss': train_results['average_loss'],
      'train_accuracy': train_results['average_accuracy'],
      'test_loss': test_results['average_loss'],
      'test_accuracy': test_results['average_accuracy']
  }
  torch.save(checkpoint, checkpoint_path)
  print(f"Checkpoint saved at {checkpoint_path}")

print("Finished Training")